In [ ]:
"""
Script Functionality:
This script processes NDWI data from the MODIS dataset for the year 2022, clipped to a region of interest (ROI) defined by a local shapefile. 
It calculates the annual median, maximum, and minimum NDWI values for the region and exports the results as GeoTIFF files to Google Drive.

Input Parameters:
- Local shapefile defining the region of interest (ROI).
- MODIS NDWI data for 2022 from the Google Earth Engine (GEE) platform.

Dependencies:
- Google Earth Engine Python API: `pip install earthengine-api`
- GeoPandas: `pip install geopandas`
- Shapely: `pip install shapely`

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

import ee  # Import the Earth Engine library
import geopandas as gpd  # Import GeoPandas for shapefile handling
from shapely.geometry import mapping  # Import mapping function for converting geometries to GeoJSON

# Initialize Google Earth Engine (authentication required)
ee.Initialize()

# Step 1: Read the local shapefile
shapefile_path = 'E:/ghana/Province/shape/ghana.shp'  # Path to the shapefile containing the region of interest (ROI)
gdf = gpd.read_file(shapefile_path)  # Load the shapefile into a GeoDataFrame

# Convert the shapefile's geometry information to GeoJSON format
roi = gdf.geometry.unary_union  # Combine all geometries into a single geometry (using unary_union)
geo_json_roi = mapping(roi)  # Convert the geometry to GeoJSON format

# Load the MODIS NDWI dataset and filter for data from the year 2022
modis_ndwi = ee.ImageCollection("MODIS/MOD09GA_006_NDWI") \
                .filterDate('2022-01-01', '2022-12-31') \
                .filterBounds(ee.Geometry.Polygon(geo_json_roi['coordinates']))  # Filter the collection to the shapefile region

# Function to clip the MODIS images to the region of interest (ROI)
def clip_to_roi(image):
    """
    Clips the given MODIS image to the region of interest (ROI).

    Args:
        image (ee.Image): The MODIS image to clip.

    Returns:
        ee.Image: The clipped image.
    """
    return image.clip(ee.Geometry.Polygon(geo_json_roi['coordinates']))  # Clip the image to the shapefile region

# Apply the clipping function to the MODIS image collection
clipped_collection = modis_ndwi.map(clip_to_roi)  # Clip all images in the collection to the ROI

# Calculate the annual median, maximum, and minimum NDWI values for the clipped region
annual_median = clipped_collection.median()  # Calculate the median NDWI for 2022
annual_max = clipped_collection.max()  # Calculate the maximum NDWI for 2022
annual_min = clipped_collection.min()  # Calculate the minimum NDWI for 2022

# Export the calculated raster data as GeoTIFF files to Google Drive
# Export the annual median NDWI image
task_median = ee.batch.Export.image.toDrive(**{
    'image': annual_median,  # Image to export
    'description': 'NDWI_2022_annual_median',  # Export task description
    'folder': 'EarthEngineExports',  # Folder in Google Drive to save the file
    'scale': 500,  # Spatial resolution (in meters)
    'region': geo_json_roi['coordinates'],  # Region to export based on the shapefile
    'fileFormat': 'GeoTIFF',  # File format
    'maxPixels': 1e13  # Large pixel limit to prevent export errors
})

# Export the annual maximum NDWI image
task_max = ee.batch.Export.image.toDrive(**{
    'image': annual_max,  # Image to export
    'description': 'NDWI_2022_annual_max',  # Export task description
    'folder': 'EarthEngineExports',  # Folder in Google Drive to save the file
    'scale': 500,  # Spatial resolution (in meters)
    'region': geo_json_roi['coordinates'],  # Region to export based on the shapefile
    'fileFormat': 'GeoTIFF',  # File format
    'maxPixels': 1e13  # Large pixel limit to prevent export errors
})

# Export the annual minimum NDWI image
task_min = ee.batch.Export.image.toDrive(**{
    'image': annual_min,  # Image to export
    'description': 'NDWI_2022_annual_min',  # Export task description
    'folder': 'EarthEngineExports',  # Folder in Google Drive to save the file
    'scale': 500,  # Spatial resolution (in meters)
    'region': geo_json_roi['coordinates'],  # Region to export based on the shapefile
    'fileFormat': 'GeoTIFF',  # File format
    'maxPixels': 1e13  # Large pixel limit to prevent export errors
})

# Start the export tasks
task_median.start()  # Start the export task for the median NDWI image
task_max.start()     # Start the export task for the maximum NDWI image
task_min.start()     # Start the export task for the minimum NDWI image

# Notify the user that the export tasks have started
print("TIFF export tasks have been started. Please check the results in your Google Drive.")

# End of the script


In [7]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.

Script Functionality:
This script resamples and aligns an NDWI raster to match the resolution, extent, and projection of a reference raster using GDAL. 
The resampled NDWI is saved as a new GeoTIFF file.

Input Parameters:
- Input NDWI raster: The NDWI raster to be resampled.
- Reference raster: An empty GeoTIFF file that serves as a template for the output resolution, extent, and projection.
- Output resampled path: The path where the resampled NDWI raster will be saved.

Dependencies:
- GDAL: `pip install gdal`

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

from osgeo import gdal  # Import the GDAL library to work with geospatial data
import os

# Enable GDAL to use exceptions for error handling
gdal.UseExceptions()

def resample_and_align(input_ndwi_path, reference_tif_path, output_resampled_path):
    """
    Resamples and aligns the input NDWI raster to match the reference raster's resolution, extent, and projection.

    Args:
        input_ndwi_path (str): Path to the NDWI raster that needs to be resampled.
        reference_tif_path (str): Path to the reference raster file.
        output_resampled_path (str): Path where the resampled NDWI will be saved.

    Returns:
        None: The function saves the resampled raster and prints a success message.
    """
    # Open the reference raster file (template for the resampled raster)
    reference_ds = gdal.Open(reference_tif_path)
    reference_proj = reference_ds.GetProjection()  # Get the projection of the reference raster
    reference_geotransform = reference_ds.GetGeoTransform()  # Get the geotransformation (coordinates and pixel resolution)
    reference_width = reference_ds.RasterXSize  # Get the width of the reference raster (number of pixels)
    reference_height = reference_ds.RasterYSize  # Get the height of the reference raster (number of pixels)

    # Use gdal.Warp to resample and align the NDWI raster
    gdal.Warp(output_resampled_path,  # Path to save the output resampled file
              input_ndwi_path,  # Input NDWI file to be resampled
              format='GTiff',  # Output format is GeoTIFF
              width=reference_width,  # Align the width to match the reference raster
              height=reference_height,  # Align the height to match the reference raster
              dstSRS=reference_proj,  # Align the coordinate system (SRS) to the reference raster
              outputBounds=(
                  reference_geotransform[0],  # xmin (left bound of the reference raster)
                  reference_geotransform[3] + reference_height * reference_geotransform[5],  # ymin (bottom bound)
                  reference_geotransform[0] + reference_width * reference_geotransform[1],  # xmax (right bound)
                  reference_geotransform[3]),  # ymax (top bound)
              resampleAlg='bilinear')  # Use bilinear interpolation for resampling

    # Notify the user that the resampling was successful
    print(f"Resampled NDWI saved to {output_resampled_path}")  # Print success message

# Input file paths
ndwi_path = "E:/ghana/NDWI/NDWI_2022_annual_median.tif"  # Path to the NDWI raster to be resampled
empty_tif_path = "E:/ghana/Raster_Templates/ghana_template/0.005.tif"  # Path to the reference empty raster file
output_resampled_path = "E:/ghana/NDWI/NDWI/NDWImedian_0.005_2022.tif"  # Path to save the resampled output

# Call the function to resample and align the NDWI raster
resample_and_align(ndwi_path, empty_tif_path, output_resampled_path)

# End of the script

Resampled NDWI saved to E:/ghana/NDWI/NDWI/NDWImedian_0.005_2022.tif
